Анализ результатов

In [77]:
import pandas as pd
import numpy as np
from typing import Dict, List, Optional, Set
from collections import Counter
import Levenshtein
import matplotlib.pyplot as plt
from IPython.core.oinspect import get_encoding

from src import future_extract as fe
from src import text_model as tm
import matplotlib as mpl
import seaborn as sns


from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
mpl.style.use('ggplot')
sns.set_style('white')
from src import preproces_text as pt
import importlib
importlib.reload(pt)
from src import optimized_model as om
importlib.reload(om)

<module 'src.optimized_model' from '/Users/vladislavdemin/PycharmProjects/BB-E-CUP/src/optimized_model.py'>

In [28]:
path_train = '/Users/vladislavdemin/Desktop/ml_hack/ml_ozon_сounterfeit_train.csv'
path_test = '/Users/vladislavdemin/Desktop/ml_hack/ml_ozon_сounterfeit_test.csv'

metadata_raw = pd.read_csv('/Users/vladislavdemin/Desktop/ml_hack/processed_data.csv')
metadata_val = pd.read_csv('/Users/vladislavdemin/Desktop/ml_hack/processed_datatest.csv')

In [29]:
data_val2 = pd.read_csv('/Users/vladislavdemin/Desktop/ml_hack/ml_ozon_сounterfeit_test.csv')

In [30]:
data_raw1, data_val1 = pt.full_preprocess(path_train, path_test)

Initialized TF-IDF + SVD (fast mode)
Processing DataFrame with 197198 rows
Text column: description
Encoding texts using fast strategy...


Encoding batches: 100%|██████████| 1972/1972 [00:31<00:00, 62.40it/s]


Created 100 embedding columns
Final DataFrame shape: (197198, 148)
Processing DataFrame with 22760 rows
Text column: description
Encoding texts using fast strategy...


Encoding batches: 100%|██████████| 228/228 [00:03<00:00, 61.59it/s]


Created 100 embedding columns
Final DataFrame shape: (22760, 147)
Processing DataFrame with 197198 rows
Text column: name_rus
Encoding texts using fast strategy...


Encoding batches: 100%|██████████| 198/198 [00:04<00:00, 42.27it/s]


Created 100 embedding columns
Final DataFrame shape: (197198, 120)
Processing DataFrame with 22760 rows
Text column: name_rus
Encoding texts using fast strategy...


Encoding batches: 100%|██████████| 23/23 [00:00<00:00, 38.86it/s]


Created 100 embedding columns
Final DataFrame shape: (22760, 120)


In [31]:
metadata_raw = metadata_raw.drop(['low_price_flag', 'ItemID', "name_rus","has_name","has_category","high_price_flag", 'id'],axis=1)
metadata_val = metadata_val.drop(['low_price_flag', 'ItemID', "name_rus","has_name","has_category","high_price_flag", 'id'],axis=1)

metadata_raw['resolution'] = data_raw1['resolution']

metadata_raw_clean, metadata_val_clean = pt.clean_small_brands(metadata_raw, metadata_val)
metadata_raw_clean_enc, metadata_val_clean_enc = pt.poluchenie_catboostenc(metadata_raw_clean, metadata_val_clean)

In [32]:
data_raw1 = data_raw1.drop(['brand_name', 'CommercialTypeName4', 'resolution'], axis=1)
data_val1 = data_val1.drop(['brand_name', 'CommercialTypeName4', ], axis=1)

conecting_data_raw = pd.concat([metadata_raw_clean_enc ,data_raw1], axis=1) 
conecting_data_val = pd.concat([metadata_val_clean_enc ,data_val1], axis=1) 

In [56]:
list(conecting_data_raw.columns)

['brand_name',
 'CommercialTypeName4',
 'rating_1_count',
 'rating_2_count',
 'rating_3_count',
 'rating_4_count',
 'rating_5_count',
 'comments_published_count',
 'photos_published_count',
 'videos_published_count',
 'PriceDiscounted',
 'item_time_alive',
 'item_count_fake_returns7',
 'item_count_fake_returns30',
 'item_count_fake_returns90',
 'item_count_sales7',
 'item_count_sales30',
 'item_count_sales90',
 'item_count_returns7',
 'item_count_returns30',
 'item_count_returns90',
 'GmvTotal7',
 'GmvTotal30',
 'GmvTotal90',
 'ExemplarAcceptedCountTotal7',
 'ExemplarAcceptedCountTotal30',
 'ExemplarAcceptedCountTotal90',
 'OrderAcceptedCountTotal7',
 'OrderAcceptedCountTotal30',
 'OrderAcceptedCountTotal90',
 'ExemplarReturnedCountTotal7',
 'ExemplarReturnedCountTotal30',
 'ExemplarReturnedCountTotal90',
 'ExemplarReturnedValueTotal7',
 'ExemplarReturnedValueTotal30',
 'ExemplarReturnedValueTotal90',
 'ItemVarietyCount',
 'ItemAvailableCount',
 'seller_time_alive',
 'SellerID',
 'bran

In [47]:
X = conecting_data_raw.drop('resolution', axis=1)
y = conecting_data_raw['resolution']  # целевая колонка


# train (70%), test (15%), val (15%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)



XGB_model1 = XGBClassifiermodel = XGBClassifier(
    n_estimators=120,
    max_depth=10,  # Уменьшаем глубину
    eval_metric='logloss',
    use_label_encoder=False,
    class_weight='balanced_subsample',
    scale_pos_weight=(len(y_train[y_train==0]) / len(y_train[y_train==1])*0.5),
    random_state=42# более мягкий вариант
)



In [62]:
XGB_model1.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              class_weight='balanced_subsample', colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='logloss', feature_types=None, feature_weights=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=120, n_jobs=None, ...)

In [63]:
threshold = 0.55
y_proba = XGB_model1.predict_proba(X_test)[:, 1]
y_pred = (y_proba >= threshold).astype(int)
accuracy_RFC_model = accuracy_score(y_test, y_pred),
report_RFC_model = classification_report(y_test, y_pred)
print(accuracy_RFC_model)
print(report_RFC_model)

(0.9777238481599536,)
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     25780
           1       0.84      0.83      0.83      1828

    accuracy                           0.98     27608
   macro avg       0.91      0.91      0.91     27608
weighted avg       0.98      0.98      0.98     27608



In [64]:
val_prediction_proba = XGB_model1.predict_proba(conecting_data_val)[:, 1]
val_prediction = (val_prediction_proba >= threshold).astype(int)

val_prediction.sum()

np.int64(2199)

In [53]:
submission = pd.DataFrame({
    'id': data_val2['id'],
    'prediction': val_prediction
    
})
submission.to_csv('/Users/vladislavdemin/Desktop/ml_hack/submission1.csv', index=False)
dddd = pd.read_csv('/Users/vladislavdemin/Desktop/ml_hack/submission1.csv')
dddd['prediction'].sum()

np.int64(2387)

In [57]:
conecting_data_raw

,brand_name,CommercialTypeName4,rating_1_count,rating_2_count,rating_3_count,rating_4_count,rating_5_count,comments_published_count,photos_published_count,videos_published_count,...,namerus_embed_90,namerus_embed_91,namerus_embed_92,namerus_embed_93,namerus_embed_94,namerus_embed_95,namerus_embed_96,namerus_embed_97,namerus_embed_98,namerus_embed_99
0,0.003009,0.001202,6.0,4.0,4.0,3.0,32.0,3.0,6.0,0.0,...,0.005238,0.018912,0.029117,0.010443,0.016382,-0.012428,0.031393,0.027076,-0.013576,-0.010717
1,0.001068,0.001003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.016369,0.012080,0.026544,-0.013873,-0.016384,0.027202,0.023666,-0.004987,0.019677,0.005069
2,0.090094,0.000164,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,-0.114257,-0.012508,0.018678,0.025701,0.063845,0.023801,0.012451,0.056973,-0.006702,-0.001221
3,0.088466,0.000028,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.030366,-0.004482,0.030858,-0.044238,-0.049003,-0.094094,-0.028146,-0.015891,-0.043423,0.007172
4,0.088466,0.000028,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.034331,-0.029953,0.062460,-0.091104,-0.037979,0.025647,0.011543,-0.051130,-0.028152,0.029630
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197193,0.019910,0.573606,4.0,1.0,0.0,6.0,60.0,76.0,29.0,9.0,...,-0.046220,-0.025159,0.000976,-0.019874,-0.006563,-0.035981,-0.063063,0.078038,-0.004156,-0.074442
197194,0.059817,0.122101,0.0,0.0,0.0,1.0,31.0,33.0,26.0,12.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
197195,0.250919,0.122101,1.0,1.0,1.0,0.0,27.0,10.0,15.0,0.0,...,0.046823,0.005686,-0.057766,0.003403,0.028806,-0.029357,-0.004244,0.007122,-0.047785,0.028454
197196,0.108964,0.467145,55.0,30.0,27.0,21.0,72.0,208.0,32.0,1.0,...,-0.004792,-0.000454,-0.027077,0.003025,0.007566,-0.001090,-0.036354,0.052352,0.002543,0.000361


In [76]:
submission = pd.DataFrame({
    'id': data_val2['id'],
    'prediction': predictions
    
})
submission.to_csv('/Users/vladislavdemin/Desktop/ml_hack/submission1.csv', index=False)
dddd = pd.read_csv('/Users/vladislavdemin/Desktop/ml_hack/submission1.csv')
dddd['prediction'].sum()

np.int64(4350)

In [78]:
# Загружаем ваши данные
detector = om.OptimizedCounterfeitDetector()

# Обучаем модель
detector.fit(
    train_df=conecting_data_raw,  # ваш train 
    test_df=conecting_data_raw,   # ваш test
    target_col='resolution'
)

# Предсказания
test_predictions = detector.predict(conecting_data_val)

# Проверка на валидации (если есть метки)
if 'resolution' in conecting_data_val.columns:
    f1 = f1_score(conecting_data_val['resolution'], test_predictions)
    print(f"Test F1: {f1:.4f}")


ПОЛНЫЙ ПАЙПЛАЙН
АНАЛИЗ ДАННЫХ

Размеры:
Train: (197198, 394)
Test: (197198, 394)

Пропуски в train (топ-10):
desc_capitalized_words_ratio    26187
desc_html_p_count               26060
desc_html_div_count             26060
desc_max_word_repetition        26060
desc_avg_paragraph_length       26060
desc_paragraph_count            26060
desc_ellipsis_count             26060
desc_question_marks             26060
desc_all_lowercase              26060
desc_all_uppercase              26060
dtype: int64

Пропуски в test (топ-10):
desc_capitalized_words_ratio    26187
desc_html_p_count               26060
desc_html_div_count             26060
desc_max_word_repetition        26060
desc_avg_paragraph_length       26060
desc_paragraph_count            26060
desc_ellipsis_count             26060
desc_question_marks             26060
desc_all_lowercase              26060
desc_all_uppercase              26060
dtype: int64

brand_name:
  Уникальных в train: 339
  Уникальных в test: 339
  Только в te

In [79]:
def cross_validate_model(train_df, target_col='resolution', n_splits=5):
    """
    Кросс-валидация для оценки стабильности модели
    """
    from sklearn.model_selection import StratifiedKFold

    X = train_df.drop(columns=[target_col])
    y = train_df[target_col]

    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    scores = []

    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
        print(f"\nFold {fold}/{n_splits}")
        print("-" * 40)

        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        # Создаем временные DataFrame
        train_fold = pd.concat([X_train, y_train], axis=1)
        val_fold = X_val.copy()

        # Обучаем модель
        detector = om.OptimizedCounterfeitDetector()
        detector.fit(train_fold, val_fold, target_col=target_col)

        # Предсказываем
        val_pred = detector.predict(X_val)
        fold_f1 = f1_score(y_val, val_pred)
        scores.append(fold_f1)

        print(f"Fold {fold} F1: {fold_f1:.4f}")

    print("\n" + "=" * 70)
    print(f"Средний F1: {np.mean(scores):.4f} (+/- {np.std(scores):.4f})")
    print("=" * 70)

    return scores


In [80]:
def train_final_model(train_path='train.csv', test_path='test.csv',
                      target_col='resolution', use_cv=False):
    """
    Обучение финальной модели
    """
    print("Загрузка данных...")
    train_df = pd.read_csv(train_path) if isinstance(train_path, str) else train_path
    test_df = pd.read_csv(test_path) if isinstance(test_path, str) else test_path

    print(f"Train shape: {train_df.shape}")
    print(f"Test shape: {test_df.shape}")

    # Опционально: кросс-валидация для оценки
    if use_cv:
        print("\nЗапуск кросс-валидации...")
        cv_scores = cross_validate_model(train_df, target_col)

        if np.mean(cv_scores) < 0.65:
            print("⚠️ Низкий средний F1 в кросс-валидации!")

    # Обучение финальной модели
    print("\nОбучение финальной модели...")
    detector = om.OptimizedCounterfeitDetector()
    detector.fit(train_df, test_df, target_col)

    # Предсказания на test
    print("\nСоздание предсказаний для test...")
    test_predictions = detector.predict(test_df)

    print(f"\nРаспределение предсказаний:")
    print(f"0 (не контрафакт): {sum(test_predictions == 0)}")
    print(f"1 (контрафакт): {sum(test_predictions == 1)}")
    print(f"Доля контрафакта: {sum(test_predictions == 1) / len(test_predictions):.3f}")

    return detector, test_predictions

In [86]:
if __name__ == "__main__":
    # Вариант 1: С вашими DataFrame
     detector, predictions = train_final_model(
         train_path=conecting_data_raw,  # ваш train DataFrame
         test_path=conecting_data_val,   # ваш test DataFrame
         target_col='resolution',
         use_cv=True  # Включить кросс-валидацию
     )
    
    
    
    
    # Сохранение результатов


Загрузка данных...
Train shape: (197198, 394)
Test shape: (22760, 393)

Запуск кросс-валидации...

Fold 1/5
----------------------------------------

ПОЛНЫЙ ПАЙПЛАЙН
АНАЛИЗ ДАННЫХ

Размеры:
Train: (157758, 394)
Test: (39440, 393)

Пропуски в train (топ-10):
desc_capitalized_words_ratio    20966
desc_html_p_count               20865
desc_html_div_count             20865
desc_max_word_repetition        20865
desc_avg_paragraph_length       20865
desc_paragraph_count            20865
desc_ellipsis_count             20865
desc_question_marks             20865
desc_all_lowercase              20865
desc_all_uppercase              20865
dtype: int64

Пропуски в test (топ-10):
desc_capitalized_words_ratio    5221
desc_html_p_count               5195
desc_html_div_count             5195
desc_max_word_repetition        5195
desc_avg_paragraph_length       5195
desc_paragraph_count            5195
desc_ellipsis_count             5195
desc_question_marks             5195
desc_all_lowercase        

In [90]:
submission = pd.DataFrame({
    'id': data_val2['id'],  # или другой ID
    'prediction': predictions
})
submission.to_csv('/Users/vladislavdemin/Desktop/ml_hack/submission1.csv', index=False)
print("\nПредсказания сохранены в submission_optimized.csv")


Предсказания сохранены в submission_optimized.csv


In [91]:
dddd = pd.read_csv('/Users/vladislavdemin/Desktop/ml_hack/submission1.csv')
dddd.sum()

id            5060458544
prediction          3628
dtype: int64